In [1]:
import numpy as np
import random
import scipy.linalg
import gym
import sys
import os
sys.path.append("../utility")
sys.path.append("../train")
sys.path.append("../")
sys.path.append("../franka/")
from utility.Utility import DerivativeLiftFunc, data_collecter,RBFLiftFunc

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
ENV = ["DampingPendulum","Pendulum-v1","CartPole-v1","MountainCarContinuous-v0","Franka","DoublePendulum"]

In [3]:
def evalKoopmanD(env_index,sample):
    env_name = ENV[env_index]
    print(env_name)
    random.seed(1)
    np.random.seed(1)
    Data_collecter = data_collecter(env_name)
    Nstate = Data_collecter.Nstates
    udim = Data_collecter.udim
    Nrbf = 50
    LiftFunc = RBFLiftFunc(env_name,Nstate,udim,Nrbf,Data_collecter.observation_space)
    NKoopman = LiftFunc.NKoopman
    test_Samples = 5000
    Steps = 30
    train_data = Data_collecter.collect_koopman_data(sample*15,1)
    X_lift = LiftFunc.Psi_su(train_data[0,:,udim:],train_data[0,:,:udim])
    Y_lift = LiftFunc.Psi_su(train_data[1,:,udim:],train_data[0,:,:udim])
    XX = np.dot(X_lift.T,X_lift)
    XY = np.dot(X_lift.T,Y_lift)
    Kd = np.dot(scipy.linalg.pinv(XX),XY).T  
    np.savez("../Prediction_Results/Samples/"+"Kd_"+env_name+"_KoopmanRBFsamples{}".format(sample)+".npz",Kd=Kd,Center=LiftFunc.center)  
    times = 4 
    max_loss_all = np.zeros((times,Steps))
    mean_loss_all = np.zeros((times,Steps))
    for t in range(times):
        test_data_path = "../Data/{}{}.npy".format(env_name,t)
        if os.path.exists(test_data_path):
            test_data = np.load("../Data/{}{}.npy".format(env_name,t))
        else:
            test_data = Data_collecter.collect_koopman_data(test_Samples,Steps)
            np.save("../Data/{}{}.npy".format(env_name,t),test_data)
        X_current = test_data[0,:,:]
        X_current_lift = np.zeros((test_Samples,NKoopman+udim))
        maxloss_list = []
        meanloss_list = []
        for i in range(test_Samples):
            X_current_lift[i] = LiftFunc.Psi_su(X_current[i,udim:],X_current[i,:udim])
        for i in range(Steps):
            X_current_lift = np.dot(X_current_lift,Kd.T)
            X_current_lift[:,NKoopman:] = test_data[i+1,:,:udim]
            Err = X_current_lift[:,:Nstate] - test_data[i+1,:,udim:]
            maxloss = np.mean(np.max(abs(Err),axis=0))
            meanloss = np.mean(np.mean(abs(Err),axis=0))
            maxloss_list.append(maxloss)
            meanloss_list.append(meanloss)
        max_loss_all[t] = np.array(maxloss_list).reshape(-1)
        mean_loss_all[t] = np.array(meanloss_list).reshape(-1)
    max_mean = np.mean(max_loss_all,axis=0)
    max_std = np.std(max_loss_all,axis=0)
    mean_mean =  np.mean(mean_loss_all,axis=0)
    mean_std =  np.std(mean_loss_all,axis=0)  
    # np.save("../Prediction_Results/Samples/"+env_name+"_KoopmanRBFsamples{}".format(sample)+".npy",np.array([max_mean,max_std,mean_mean,mean_std]))
    np.save("../Prediction_Results/Compare/"+env_name+"_KoopmanRBF"+".npy",np.array([max_mean,max_std,mean_mean,mean_std]))
    return max_mean,max_std,mean_mean,mean_std 

In [5]:
# for sample in [200, 1000, 5000, 20000, 50000]:
sample = 20000
# for i in range(6):
evalKoopmanD(4,sample)
# for sample in [200]:
#      evalKoopmanD(1,sample)
# for sample in [200]:
#      evalKoopmanD(2,sample)
# for sample in [200]:
#      evalKoopmanD(3,sample)
# for sample in [200]:
#      evalKoopmanD(4,sample)
# for sample in [200]:
#      evalKoopmanD(5,sample)

Franka
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
panda_link0

pybullet build time: Jan 29 2025 23:20:52


b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
panda_link1b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
panda_link2b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
panda_link3b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfIm

(array([3.66832662e-04, 3.60426422e-01, 7.44691762e-01, 1.40097564e+00,
        3.06260759e+00, 6.91715184e+00, 1.60919542e+01, 3.73895582e+01,
        8.61455731e+01, 1.97021216e+02, 4.48871785e+02, 1.01950983e+03,
        2.31127722e+03, 5.23279931e+03, 1.18374396e+04, 2.67631566e+04,
        6.04863092e+04, 1.36669757e+05, 3.08758237e+05, 6.97459109e+05,
        1.57539197e+06, 3.55826617e+06, 8.03664708e+06, 1.81510820e+07,
        4.09943746e+07, 9.25853159e+07, 2.09101606e+08, 4.72248843e+08,
        1.06655509e+09, 2.40876771e+09]),
 array([1.92738187e-05, 3.04810738e-03, 6.04400243e-03, 3.08626621e-02,
        9.45493270e-02, 2.65929033e-01, 7.00566755e-01, 1.62322703e+00,
        3.96750755e+00, 9.34381111e+00, 2.15031502e+01, 4.92384062e+01,
        1.12318204e+02, 2.55161675e+02, 5.78706695e+02, 1.31052590e+03,
        2.96516529e+03, 6.70458527e+03, 1.51537393e+04, 3.42416951e+04,
        7.73596111e+04, 1.74752302e+05, 3.94728437e+05, 8.91563024e+05,
        2.01368265e+06